In [3]:
import keras
from keras.layers import Input, Dense
from keras.models import Model,Sequential
from sklearn.metrics import classification_report,confusion_matrix
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.svm import SVC
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

ImportError: No module named 'tensorflow.contrib'

In [3]:
import tensorflow as tf
hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))



AttributeError: module 'tensorflow' has no attribute 'Session'

In [ ]:
#      -- Only 14 used
#      -- 1. #3  (age)           age in years
#      -- 2. #4  (sex)           sex (1 = male; 0 = female)
#      -- 3. #9  (cp)        chest pain type
#                                -- Value 1: typical angina
#                                -- Value 2: atypical angina
#                                -- Value 3: non-anginal pain
#                                -- Value 4: asymptomatic
#      -- 4. #10 (trestbps)  resting blood pressure (in mm Hg on admission to the hospital)
#      -- 5. #12 (chol)      serum cholestoral in mg/dl
#      -- 6. #16 (fbs)       (fasting blood sugar > 120 mg/dl)  (1 = true; 0 = false)
#      -- 7. #19 (restecg)   resting electrocardiographic results
#                                -- Value 0: normal
#                                -- Value 1: having ST-T wave abnormality (T wave inversions and/or ST 
#                                            elevation or depression of > 0.05 mV)
#                                -- Value 2: showing probable or definite left ventricular hypertrophy
#                                            by Estes' criteria
#      -- 8. #32 (thalach)   maximum heart rate achieved
#      -- 9. #38 (exang)     exercise induced angina (1 = yes; 0 = no)
#      -- 10. #40 (oldpeak)  ST depression induced by exercise relative to rest
#      -- 11. #41 (slope)    the slope of the peak exercise ST segment
#                                -- Value 1: upsloping
#                                -- Value 2: flat
#                                -- Value 3: downsloping
#      -- 12. #44 (ca)       number of major vessels (0-3) colored by flourosopy
#      -- 13. #51 (thal)     3 = normal; 6 = fixed defect; 7 = reversable defect
#      -- 14. #58 (num)      diagnosis of heart disease (angiographic disease status)
#                                -- Value 0: < 50% diameter narrowing
#                                -- Value 1: > 50% diameter narrowing
#                                (in any major vessel: attributes 59 through 68 are vessels)

In [ ]:
dataset = pd.read_csv('C:\\Users\\Devesh.Raj\\OneDrive - Unilever\\Study\\IITH\\MDS_Project\\heart.csv')

In [ ]:
dataset.head()

In [ ]:
#Data Exploration
dataset.target.value_counts()

In [ ]:
sns.countplot(x="target", data=dataset, palette="bwr")
plt.show()

In [ ]:
sns.countplot(x='sex', data=dataset, palette="mako_r")
plt.xlabel("Sex (0 = female, 1= male)")
plt.show()


In [ ]:
dataset.groupby('target').mean()

In [ ]:
pd.crosstab(dataset.age,dataset.target).plot(kind="bar",figsize=(20,6))
plt.title('Heart Disease Frequency for Ages')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.savefig('heartDiseaseAndAges.png')
plt.show()

In [ ]:
pd.crosstab(dataset.sex,dataset.target).plot(kind="bar",figsize=(15,6),color=['#1CA53B','#AA1111' ])
plt.title('Heart Disease Frequency for Sex')
plt.xlabel('Sex (0 = Female, 1 = Male)')
plt.xticks(rotation=0)
plt.legend(["Not Have Disease", "Have Disease"])
plt.ylabel('Frequency')
plt.show()


In [ ]:
plt.scatter(x=dataset.age[dataset.target==1], y=dataset.thalach[(dataset.target==1)], c="red")
plt.scatter(x=dataset.age[dataset.target==0], y=dataset.thalach[(dataset.target==0)])
plt.legend(["Disease", "Not Disease"])
plt.xlabel("Age")
plt.ylabel("Maximum Heart Rate")
plt.show()

In [ ]:
pd.crosstab(dataset.slope,dataset.target).plot(kind="bar",figsize=(15,6),color=['#DAF7A6','#FF5733' ])
plt.title('Heart Disease Frequency for Slope')
plt.xlabel('The Slope of The Peak Exercise ST Segment ')
plt.xticks(rotation = 0)
plt.ylabel('Frequency')
plt.show()

In [ ]:
pd.crosstab(dataset.fbs,dataset.target).plot(kind="bar",figsize=(15,6),color=['#FFC300','#581845' ])
plt.title('Heart Disease Frequency According To FBS')
plt.xlabel('FBS - (Fasting Blood Sugar > 120 mg/dl) (1 = true; 0 = false)')
plt.xticks(rotation = 0)
plt.legend(["Haven't Disease", "Have Disease"])
plt.ylabel('Frequency of Disease or Not')
plt.show()

In [ ]:
pd.crosstab(dataset.cp,dataset.target).plot(kind="bar",figsize=(15,6),color=['#11A5AA','#AA1190' ])
plt.title('Heart Disease Frequency According To Chest Pain Type')
plt.xlabel('Chest Pain Type')
plt.xticks(rotation = 0)
plt.ylabel('Frequency of Disease or Not')
plt.show()


# Creating Dummy Variables

In [ ]:
X = dataset.drop(['target'], axis = 1)
y = dataset['target']
cat_feats = ['cp','restecg','slope','ca','thal']
bin_feats = ['sex','fbs','exang']
numeric_feats = ['age','trestbps','chol','thalach','oldpeak']
X = pd.get_dummies(X,columns=cat_feats,drop_first = True)


In [ ]:
X.head()

In [ ]:
Sc = StandardScaler()
# scaling only numerical features
for feature in numeric_feats:
    X[[feature]] = Sc.fit_transform(X[[feature]])

In [ ]:
X.head()

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state = 101)

In [ ]:
X.shape

In [ ]:

input_dim = 22
# this is the size of our encoded representations
encoding_dim_layer = 22

# this is our input placeholder
input_data = Input(shape=(input_dim,))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim_layer, activation="linear")(input_data)
decoded = Dense(input_dim,activation='linear')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_data, decoded)

# this model maps an input to its encoded representation
encoder = Model(input_data, encoded)

# create a placeholder for an encoded (x-dimensional) input
encoded_input = Input(shape=(encoding_dim_layer,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

autoencoder.compile(optimizer='adam', loss='mean_absolute_error')

In [ ]:
autoencoder.fit(X.values, X.values,
                epochs=300,
                batch_size=30,
                shuffle=True,verbose=0)

In [ ]:
encoded_data_train = encoder.predict(X_train.values)
encoded_data_test = encoder.predict(X_test.values)
decoded_data_train = decoder.predict(encoded_data_train)
decoded_data_test = decoder.predict(encoded_data_test)

In [ ]:
X_train.head(1)

In [ ]:

decoded_data_train[0]

In [ ]:
#Create the neural network for Encoded Data
model = Sequential()
model.add(Dense(units = int(encoding_dim_layer/2),activation = 'tanh',kernel_initializer = 'uniform',input_dim = encoding_dim_layer))
model.add(Dense(units = 1,activation = 'sigmoid',kernel_initializer = 'uniform'))
model.compile(optimizer= 'adam',metrics = ['accuracy'],loss = 'binary_crossentropy')
model.fit(encoded_data_train,y_train,batch_size = 20,epochs=100,verbose=0)

In [ ]:
X_train_1 = pd.DataFrame(X_train.values)
X_test_1 = pd.DataFrame(X_test.values)

In [ ]:
X_encoded_train = pd.DataFrame(encoded_data_train)
X_encoded_test = pd.DataFrame(encoded_data_test)

In [ ]:
#Club the encoded and input features
for i in range(0,encoding_dim_layer):
    X_train_1[i+input_dim] = X_encoded_train[i]
    X_test_1[i+input_dim] = X_encoded_test[i]

In [ ]:
X_train_1.columns

In [ ]:
# SVC for original input features
svc = SVC(C=1,kernel='linear',probability=True)
svc.fit(X_train,y_train)
predictions1 = svc.predict_proba(X_test)[:,1] > 0.5
print("###### Results from original features Using SVC #########\n")
print(confusion_matrix(y_test,predictions1))
print(classification_report(y_test,predictions1))

In [ ]:
 #SVC for the clubbed features

svc = SVC(C=1,kernel='linear',probability=True)
svc.fit(X_train_1,y_train)
predictions5 = svc.predict_proba(X_test_1)[:,1] > 0.5
print("###### Results from Clubbed Features Using SVC #########\n")
print(confusion_matrix(y_test,predictions5))
print(classification_report(y_test,predictions5))

# Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
grid_param = {'C':[0.01,0.1,1,10,100],'gamma':[1,0.1,0.001,0.0001],'kernel':['rbf','linear']}

grid = GridSearchCV(SVC(probability=True),grid_param,n_jobs = -1,verbose = 2)

In [ ]:
grid.fit(X_train,y_train)

In [ ]:
grid.best_estimator_

In [ ]:
grid.best_params_

In [ ]:
predictions2 = grid.predict_proba(X_test)[:,1] > 0.5
print("###### Results from original features Using hyper parameter optimization SVC #########\n")
print(confusion_matrix(y_test,predictions2))
print(classification_report(y_test,predictions2))

In [ ]:
print(confusion_matrix(y_test,grid.predict(X_test)))
print(classification_report(y_test,grid.predict(X_test)))